![image.png](https://i.imgur.com/a3uAqnb.png)

## 💡 **Introduction**

In this lab, we explore how to build, train, and evaluate a **neural network for image classification** using **PyTorch**.  
Our goal is to classify handwritten digits (0-9) from the **MNIST dataset** using PyTorch's powerful deep learning framework.  
Through this hands-on exercise, we demonstrate the complete workflow of building neural networks with modern tools — from model architecture to training loops and GPU acceleration.

The dataset consists of **28×28 grayscale images** of handwritten digits, where each pixel value ranges from 0 to 255.  

#### 🎯 **What We'll Build**
- **1-Layer Neural Network:** Direct mapping from input to output with softmax activation
- **2-Layer Neural Network:** Input → Hidden Layer (ReLU) → Output (Softmax)

#### 🔑 **Key PyTorch Concepts**
- **Automatic Differentiation:** PyTorch handles backpropagation automatically
- **GPU Acceleration:** Train models faster using CUDA-enabled GPUs
- **Modular Design:** Build reusable model classes with `nn.Module`
- **Built-in Optimizers:** Use Adam optimizer for efficient weight updates
- **Cross-Entropy Loss:** Standard loss function for multi-class classification

By leveraging PyTorch's high-level abstractions, we can focus on model design while the framework handles the computational complexities.

### 🧠 **Importing Libraries and Frameworks**

In this step, we import all the essential Python libraries for our PyTorch-based neural network implementation:

- **PyTorch** (`torch`) — the core deep learning framework for tensor operations and automatic differentiation.  
- **torch.nn** — contains neural network layers, activation functions, and loss functions.  
- **torch.optim** (`Adam`) — provides optimization algorithms for updating model weights.  
- **torchvision.datasets** (`MNIST`) — for easily downloading and loading the MNIST dataset.  
- **torch.utils.data** (`DataLoader`) — for creating batches of data efficiently during training.  
- **torchvision.transforms.functional** (`to_tensor`) — to convert PIL images to tensor format.  
- **Matplotlib** (`plt`) — for visualizing training curves and results.  
- **IPython.display** (`clear_output`) — to clean up console output for better readability.

These imports provide everything we need to build, train, and evaluate our neural network using PyTorch's powerful ecosystem.

In [ ]:
from IPython.display import clear_output

In [ ]:
# Download the required libraries (needed when running outside colab where the environment doesn't come pre-loaded with libraries)

%pip install torch
%pip install matplotlib
%pip install torchvision

clear_output()

In [ ]:
import torch

import torch.nn as nn

from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torch.optim import Adam

from torchvision.transforms.functional import to_tensor

import matplotlib.pyplot as plt

### 📋 **Lab Contents and Overview**

In this notebook, we'll implement **1-layer and 2-layer neural networks** in PyTorch for classifying the MNIST dataset.

#### 📷 **About MNIST**
The dataset consists of **28×28 grayscale images**, each containing a handwritten digit from 0 to 9.  
Our model needs to take these images and classify them to the correct digit.

#### 🛠️ **Prerequisites**
**Required Knowledge:**
1. **PyTorch** fundamentals (tensors, autograd, nn.Module)
2. **Matplotlib** basics (for visualization)

**Good to Have:**
1. Understanding of `torch.utils.data.Dataset` and `DataLoader`
2. Familiarity with GPU computing concepts (CUDA)

#### 🎓 **Learning Objectives**
By the end of this lab, you will:
- Build custom neural network architectures using `nn.Module`
- Train models with automatic differentiation
- Use GPU acceleration for faster training
- Implement proper train/validation loops
- Save and load trained models

Let's get started!

### 📊 **Loading and Preparing the MNIST Dataset**

In this section, we load and prepare the MNIST dataset for training and testing our neural network.

#### 1️⃣ **Dataset Loading**
- The `MNIST` function from `torchvision.datasets` automatically downloads the dataset if not already present.
- **Training set:** 60,000 images
- **Test set:** 10,000 images

#### 2️⃣ **Data Transformation**
- `transform=to_tensor` converts PIL images to PyTorch tensors
- This transformation:
  - Converts pixel values from [0, 255] to [0.0, 1.0]
  - Changes format from (H, W, C) to (C, H, W) — PyTorch's expected format
  - Enables GPU acceleration and automatic differentiation

#### 3️⃣ **Important Note**
We're using PyTorch's data utilities for dataset handling, which provides:
- Automatic downloading and caching
- Easy batching with `DataLoader`
- Efficient memory management
- Support for parallel data loading

The images will later be flattened from (1, 28, 28) to (784,) when fed into our fully connected layers.

In [ ]:
# MNIST function fetches the MNIST dataset. Without any transform param, the returned object is a Pillow image but we want to convert it to numerical form
# that is to say, a numpy array/torch tensor

# to_tensor is used to avoid errors when creating data loader later. we'll convert them to numpy arrays when the time comes
train_data = MNIST(root='./datasets', train=True, download=True, transform=to_tensor)
test_data  = MNIST(root='./datasets', train=False, download=True, transform=to_tensor)

clear_output()

### 🔄 **Creating DataLoaders for Batch Processing**

DataLoaders are essential PyTorch utilities that handle batching, shuffling, and efficient data loading during training.

#### 📦 **Why Use DataLoaders?**
- **Batch Gradient Descent:** Process multiple samples simultaneously for faster convergence
- **Memory Efficiency:** Load data in chunks rather than all at once
- **Automatic Shuffling:** Randomize data order each epoch (when `shuffle=True`)
- **Parallel Loading:** Use multiple workers for faster data preprocessing (via `num_workers` parameter)

#### ⚙️ **Configuration**
- **Batch Size:** 64 samples per batch
  - Larger batches → faster training but more memory usage
  - Smaller batches → more stable gradients but slower training

The DataLoader will automatically:
1. Group samples into batches of size 64
2. Return tensors of shape `(64, 1, 28, 28)` for images
3. Return tensors of shape `(64,)` for labels

This makes our training loop clean and efficient!

In [ ]:
batch_size = 64

# Dataloaders are used to easily create batches of data so we can perform batch gradient descent for faster learning
train_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

### 🏗️ **Model Architectures: 1-Layer vs 2-Layer Networks**

Let's create the architecture of our models using PyTorch's `nn.Module` class.

#### 🔍 **Softmax vs Sigmoid**
Instead of sigmoid activation (used in NumPy implementation), we'll use **Softmax** activation:

- **Sigmoid:** Maps each value independently to range [0, 1]
  - Output values don't sum to 1
  - Each output is independent
  
- **Softmax:** Converts logits into a probability distribution
  - Output values sum to 1
  - Highest input value remains highest in output
  - Perfect for multi-class classification

**Softmax Formula:**
$$
\text{Softmax}(x)_i = \frac{e^{x_i}}{\sum_{j} e^{x_j}}
$$

#### 📐 **Model 1: Single-Layer Network**
Input (784) → Linear → Softmax → Output (10)


- **Simplest possible classifier**
- Learns direct mapping from pixels to classes
- No hidden representations
- Limited expressiveness but fast to train

#### 📐 **Model 2: Two-Layer Network**

- **Hidden layer** learns intermediate features
- **ReLU activation** introduces non-linearity
- More expressive than single layer
- Comparable to our NumPy implementation


#### 🧩 **PyTorch Module Structure**
Each model inherits from `nn.Module` and must implement:
1. **`__init__()`** — Define layers and operations
2. **`forward()`** — Specify forward pass computation

PyTorch automatically handles:
- Backpropagation (via autograd)
- Parameter management
- GPU transfer
- Gradient computation

In [ ]:
class NN1Layer(nn.Module):
  """
  Single-layer neural network for MNIST classification.
  Architecture: Input → Linear → Softmax → Output
  """
  def __init__(self, num_inp, num_out):
    
    super(NN1Layer, self).__init__()
    
    # Single fully connected layer
    self.layer_1 = nn.Linear(num_inp, num_out)
    self.softmax = nn.Softmax(dim=1)  # Apply softmax across class dimension
  
  def forward(self, x):
    
    z = self.layer_1(x)
    a = self.softmax(z)
    
    return a


class NN2Layer(nn.Module):
  """
  Two-layer neural network for MNIST classification.
  Architecture: Input → Linear → ReLU → Linear → Softmax → Output
  """
  def __init__(self, num_inp, num_hidden, num_out):
    
    super(NN2Layer, self).__init__()
    
    # Define layers
    self.layer_1 = nn.Linear(num_inp, num_hidden)
    self.layer_2 = nn.Linear(num_hidden, num_out)
    
    # Activation functions
    self.hidden_activation = nn.ReLU()  # We can change the hidden activation here (try Sigmoid, Tanh, etc.)
    self.softmax = nn.Softmax(dim=1)    # dim 0 is batch size, we apply softmax across classes (dim 1)
  
  def forward(self, x):
    
    # Hidden layer
    z1 = self.layer_1(x)
    a1 = self.hidden_activation(z1)
    
    # Output layer
    z2 = self.layer_2(a1)
    a2 = self.softmax(z2)
    
    return a2

### ⚙️ **Training Setup: Model, Optimizer, and Device Configuration**

Before training, we need to configure our model, optimizer, loss function, and compute device.

#### 🎛️ **Hyperparameters**
- **Number of Epochs:** 12 — how many times to iterate through the entire dataset
- **Learning Rate:** 1e-4 (0.0001) — controls step size during optimization
  - Small learning rate → stable but slow training
  - Large learning rate → fast but may overshoot optimal values

#### 🖥️ **Device Selection (CPU vs GPU)**
```python
device = 'cuda' if torch.cuda.is_available() else 'cpu'
```
- **CUDA (GPU):** Dramatically faster training via parallel computation
- **CPU:** Works everywhere but significantly slower
- All tensors and the model must be moved to the same device

**Pro Tip:** Change `device` from `'cuda'` to `'cpu'` and re-run to see the performance difference!

#### 🏗️ **Model Selection**
We'll use the **2-layer network** (`NN2Layer`) which is equivalent to our NumPy implementation:
- Input size: 784 (28×28 flattened)
- Hidden layer: 32 neurons
- Output size: 10 classes

You can comment/uncomment to switch between 1-layer and 2-layer models.

#### 📉 **Loss Function**
- **CrossEntropyLoss:** Combines softmax + negative log-likelihood
- Ideal for multi-class classification
- PyTorch's implementation is numerically stable

#### 🔧 **Optimizer**
- **Adam:** Adaptive learning rate optimizer
- Combines benefits of momentum and RMSProp
- Generally works well with minimal tuning

#### 📊 **Tracking**
We maintain lists to store training and validation losses for visualization later.

In [ ]:
num_epochs = 12
lr = 1e-4

device = 'cuda' if torch.cuda.is_available() else 'cpu'  # checks if machine supports cuda and if it does, we use that, otherwise cpu

train_losses = []
val_losses = []

# Choose model architecture
# model = NN1Layer(28*28, 10)  # Single-layer: 28*28 input, 10 output classes (0-9)
model = NN2Layer(28*28, 32, 10)  # Two-layer: equivalent to NumPy implementation

# Configure optimizer and loss function
optimizer = Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()  # Multi-class classification loss

# Move model to GPU/CPU
model.to(device)  # All input tensors must also be moved to this device

print(f'Using device: {device}')

### 🔍 **Pre-Training Performance Evaluation**

Before training, let's evaluate the model's initial (random) performance on the test set.

#### 🎯 **Purpose**
- Establish a **baseline** to compare against after training
- Verify that the model and data pipeline work correctly
- For random initialization, we expect ~10% accuracy (random guessing for 10 classes)

#### ⚙️ **Evaluation Process**
1. **Set model to evaluation mode:** `model.eval()`
   - Disables dropout and batch normalization training behaviors
   
2. **Disable gradient computation:** `torch.no_grad()`
   - Saves memory and speeds up inference
   - Gradients aren't needed during evaluation

3. **Compute metrics:**
   - **Loss:** Average cross-entropy over all test samples
   - **Accuracy:** Percentage of correctly classified images

This gives us a clear picture of the untrained model's performance before any learning occurs.

In [ ]:
# Evaluate untrained model on test set

model.eval()  # Set to evaluation mode
correctly_labelled = 0

with torch.no_grad():  # Disable gradient tracking for efficiency
  
  val_epoch_weighted_loss = 0
  
  for val_batch_X, val_batch_y in test_loader:
    
    # Move data to device and reshape
    val_batch_X = val_batch_X.view(-1, 28*28).to(device)
    val_batch_y = val_batch_y.to(device)
    
    # Forward pass
    val_batch_y_probs = model(val_batch_X)
    
    # Calculate loss
    loss = criterion(val_batch_y_probs, val_batch_y)
    val_epoch_weighted_loss += (len(val_batch_y) * loss.item())
    
    # Calculate accuracy
    val_batch_y_pred = val_batch_y_probs.argmax(dim=1)  # Get class with highest probability
    correctly_labelled += (val_batch_y_pred == val_batch_y).sum().item()

  val_epoch_loss = val_epoch_weighted_loss / len(test_loader.dataset)

print(f'Pre-training: val_loss={val_epoch_loss:.4f}, {correctly_labelled}/{len(test_loader.dataset)} correctly labelled ({correctly_labelled/len(test_loader.dataset)*100:.2f}% accuracy)')

### 🚀 **Training Loop: Full Model Training with Validation**

Now we train the neural network for multiple epochs using **batch gradient descent** with PyTorch's automatic differentiation.

#### 🔄 **Training Process (Per Epoch)**

**📚 Training Phase:**
1. **Set model to training mode:** `model.train()`
   - Enables dropout, batch normalization, and other training-specific layers

2. **For each batch:**
   - Move data to device (GPU/CPU)
   - Reshape images from (batch_size, 1, 28, 28) to (batch_size, 784)
   - **Forward pass:** Compute predictions
   - **Loss computation:** Calculate cross-entropy loss
   - **Backward pass:** `loss.backward()` — automatic gradient computation!
   - **Optimization:** `optimizer.step()` — update weights
   - **Gradient reset:** `optimizer.zero_grad()` — clear old gradients

3. **Track training loss** across all batches

**✅ Validation Phase:**
1. **Set model to evaluation mode:** `model.eval()`
   
2. **Disable gradient tracking:** `torch.no_grad()`
   - Saves memory and speeds up evaluation

3. **For each test batch:**
   - Compute predictions and loss
   - Calculate accuracy by comparing predicted vs true labels
   - Track validation loss

4. **Print epoch metrics:**
   - Training loss, validation loss, and accuracy

---

#### ⚡ **GPU Acceleration**
- The `%%time` magic command measures total training time
- **Experiment:** Change `device` variable to `'cpu'` and re-run to see the speedup from GPU!

---

#### 📊 **Expected Behavior**
- **Training loss** should decrease steadily
- **Validation loss** should also decrease
- **Accuracy** should improve (aim for >95%)
- If validation loss increases while training loss decreases → **overfitting**

In [ ]:
%%time
for epoch_no in range(num_epochs):
  
  # ============ TRAINING PHASE ============
  model.train()  # Enable training-specific layers (dropout, batch norm, etc.)
  
  epoch_weighted_loss = 0
  
  for batch_X, batch_y in train_loader:
    
    # Move data to device and reshape
    batch_X = batch_X.view(-1, 28*28).to(device)  # Flatten: (batch_size, 1, 28, 28) → (batch_size, 784)
    batch_y = batch_y.to(device)
    
    # Forward pass
    batch_y_probs = model(batch_X)  # Output: (batch_size, 10) — probabilities for each class
    
    # Compute loss
    loss = criterion(batch_y_probs, batch_y)
    
    # Backward pass and optimization
    optimizer.zero_grad()  # Clear gradients from previous batch
    loss.backward()        # Compute gradients via backpropagation
    optimizer.step()       # Update weights using gradients
    
    # Track loss
    epoch_weighted_loss += (len(batch_y) * loss.item())
  
  # Average training loss for epoch
  epoch_loss = epoch_weighted_loss / len(train_loader.dataset)
  train_losses.append(epoch_loss)
  
  
  # ============ VALIDATION PHASE ============
  model.eval()  # Disable training-specific layers
  correctly_labelled = 0
  
  with torch.no_grad():  # Disable gradient tracking for efficiency
    
    val_epoch_weighted_loss = 0
    
    for val_batch_X, val_batch_y in test_loader:
      
      # Move data to device and reshape
      val_batch_X = val_batch_X.view(-1, 28*28).to(device)
      val_batch_y = val_batch_y.to(device)
      
      # Forward pass
      val_batch_y_probs = model(val_batch_X)
      
      # Compute loss
      loss = criterion(val_batch_y_probs, val_batch_y)
      val_epoch_weighted_loss += (len(val_batch_y) * loss.item())
      
      # Calculate predictions and accuracy
      val_batch_y_pred = val_batch_y_probs.argmax(dim=1)  # Convert probabilities to class labels
      correctly_labelled += (val_batch_y_pred == val_batch_y).sum().item()
    
    # Average validation loss for epoch
    val_epoch_loss = val_epoch_weighted_loss / len(test_loader.dataset)
    val_losses.append(val_epoch_loss)
  
  # Print epoch summary
  print(f'Epoch {epoch_no+1}/{num_epochs}: train_loss={epoch_loss:.4f}, val_loss={val_epoch_loss:.4f}, accuracy={correctly_labelled}/{len(test_loader.dataset)} ({correctly_labelled/len(test_loader.dataset)*100:.2f}%)')

print(f'\n✅ Training complete on device: {device}')
print('💡 Tip: Change device variable to "cpu" and re-run to compare performance!')

### 📈 **Visualizing Training Progress: Loss Trajectory**

After training, we plot the **training and validation loss curves** to understand the model's learning behavior.

#### 🔍 **Interpretation**

**✅ Good Signs:**
- Both losses **decrease** over epochs
- Curves are **relatively smooth**
- Training and validation losses stay **close together**
- Model has **converged** (losses plateau)

**⚠️ Warning Signs:**
- **Validation loss increases** while training loss decreases → **Overfitting**
  - Model memorizes training data but doesn't generalize
  - Solutions: Add regularization, dropout, or early stopping
  
- **Both losses remain high** → **Underfitting**
  - Model too simple or learning rate too small
  - Solutions: Increase model capacity, adjust learning rate

- **Noisy/erratic curves** → Training instability
  - Solutions: Reduce learning rate, increase batch size

This visualization is crucial for diagnosing training issues and understanding model performance!

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Train Loss', linewidth=2, marker='o')
plt.plot(val_losses, label='Validation Loss', linewidth=2, marker='s')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss (Cross Entropy)', fontsize=12)
plt.title('Training Curve — MNIST Digit Classification (PyTorch)', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

### 🔢 **Understanding Tensor Shapes and Softmax Behavior**

Let's examine the shapes of our tensors and verify that softmax works as expected.

#### 📐 **Tensor Dimensions**
- **Input:** `(batch_size, 784)` — flattened images
- **Predictions (probabilities):** `(batch_size, 10)` — probability for each of 10 classes
- **Labels (predicted):** `(batch_size,)` — single class index per sample

#### 🔍 **Verifying Softmax**
Softmax should convert logits into a **probability distribution**:
- Each row sums to **1.0**
- Each value is between **0.0 and 1.0**
- Highest logit becomes highest probability

#### 🎯 **Argmax in Action**
`argmax(dim=1)` finds the **index** of the maximum value:
- Input: `[0.01, 0.05, 0.02, 0.87, 0.01, ...]` (probabilities)
- Output: `3` (index of 0.87)

This converts probability distributions into concrete class predictions!

In [ ]:
print("=" * 50)
print("TENSOR SHAPES")
print("=" * 50)

print(f"Input shape (flattened images): {val_batch_X.shape}")
print(f"True labels shape: {val_batch_y.shape}")
print(f"Predicted probabilities shape: {val_batch_y_probs.shape}")
print(f"Predicted labels shape: {val_batch_y_pred.shape}")

print("\n" + "=" * 50)
print("VERIFYING SOFTMAX (probabilities should sum to 1)")
print("=" * 50)

# Sum probabilities across classes (dim=1)
prob_sums = val_batch_y_probs.sum(1)
print(f"\nSum of probabilities for each sample:\n{prob_sums}")
print(f"\nShape of sums: {prob_sums.shape}")

print("\n" + "=" * 50)
print("ARGMAX IN ACTION")
print("=" * 50)

test_idx = 15
print(f"\nSample {test_idx} probabilities (10 classes):")
print(val_batch_y_probs[test_idx].cpu())  # .cpu() moves tensor from GPU to CPU for display

print(f"\nPredicted class (argmax):")
print(val_batch_y_pred[test_idx].cpu())

print(f"\nTrue class:")
print(val_batch_y[test_idx].cpu())

### 💾 **Saving and Loading Trained Models**

PyTorch makes it easy to save trained models and load them later for inference or continued training.

#### 📦 **Saving a Model**
```python
torch.save(model.state_dict(), 'model_name.pt')
```
- **`state_dict()`** contains all model parameters (weights and biases)
- Saved as a dictionary mapping parameter names to tensors
- File extension `.pt` or `.pth` is standard convention

#### 📂 **Loading a Model**
```python
model = ModelClass(...)          # Create model with same architecture
model.load_state_dict(torch.load('model_name.pt'))
model.eval()                     # Set to evaluation mode
```

#### ⚠️ **Important Notes**
- Must create model with **identical architecture** before loading
- Use `model.eval()` after loading for inference
- Move model to correct device: `model.to(device)`

#### 🧪 **Verification**
We'll demonstrate by:
1. Creating a fresh (untrained) model → poor performance
2. Loading saved weights → excellent performance

This proves the weights were successfully saved and restored!

In [ ]:
# Save the trained model's state dictionary
torch.save(model.state_dict(), 'MNIST_classifier.pt')

print("✅ Model saved successfully as 'MNIST_classifier.pt'")
print("📁 Check your storage/file system to see the saved file")

### 🔄 **Demonstrating Model Loading**

Let's verify that our saved model can be loaded and produces the same results.

#### 🧪 **Experiment Design**
1. **Create untrained model** with random weights
   - Expected: Poor performance (~10% accuracy)

2. **Load trained weights** from saved file
   - Expected: Excellent performance (>95% accuracy)

This demonstrates:
- Models start with random initialization
- Saved weights perfectly restore trained performance
- Model persistence works correctly

#### 💡 **Practical Use Cases**
- **Deployment:** Load trained model for production inference
- **Transfer Learning:** Load pre-trained weights and fine-tune
- **Checkpointing:** Save during training to resume later
- **Sharing:** Distribute trained models to others

In [ ]:
# Create a fresh model with random initialization
loaded_model = NN2Layer(28*28, 32, 10)
loaded_model.eval().to(device)

print("=" * 60)
print("BEFORE LOADING TRAINED WEIGHTS (Random Initialization)")
print("=" * 60)

with torch.no_grad():
  probs = loaded_model(val_batch_X)
  preds = probs.argmax(dim=1)
  
  correct = (preds == val_batch_y).sum()
  total = len(preds)
  accuracy = (correct / total * 100).item()
  
  print(f"Accuracy: {correct}/{total} correct ({accuracy:.2f}%)")
  print("Expected: ~10% (random guessing for 10 classes)")

print("\n" + "=" * 60)
print("AFTER LOADING TRAINED WEIGHTS")
print("=" * 60)

# Load the saved weights
loaded_model.load_state_dict(torch.load('MNIST_classifier.pt'))

with torch.no_grad():
  probs = loaded_model(val_batch_X)
  preds = probs.argmax(dim=1)
  
  correct = (preds == val_batch_y).sum()
  total = len(preds)
  accuracy = (correct / total * 100).item()
  
  print(f"Accuracy: {correct}/{total} correct ({accuracy:.2f}%)")
  print("Expected: >95% (trained model performance)")

print("\n✅ Model successfully loaded and verified!")

## 🧾 **Summary**

In this experiment, we successfully built and trained neural networks for MNIST digit classification using PyTorch.

#### 🎯 **What We Accomplished**

1. **Loaded and preprocessed** the MNIST dataset using `torchvision` and `DataLoader`

2. **Implemented two neural network architectures:**
   - **1-Layer Network:** Direct input-to-output mapping
   - **2-Layer Network:** With hidden layer and ReLU activation

3. **Trained models** using:
   - **Automatic differentiation** (no manual gradient computation!)
   - **Adam optimizer** for efficient weight updates
   - **Cross-entropy loss** for multi-class classification

4. **Leveraged GPU acceleration** for significantly faster training

5. **Visualized training progress** through loss curves

6. **Saved and loaded models** for persistence and deployment

---

#### 🔑 **Key Takeaways**

✅ **PyTorch Advantages:**
- **Automatic differentiation** eliminates manual backpropagation
- **GPU support** dramatically speeds up training
- **Modular design** with `nn.Module` creates reusable components
- **Rich ecosystem** with pre-built layers, optimizers, and utilities

✅ **Best Practices:**
- Always move model and data to the same device (`cuda` or `cpu`)
- Use `model.train()` during training and `model.eval()` during evaluation
- Track both training and validation losses to detect overfitting
- Save models regularly for checkpointing and deployment

✅ **Performance:**
- Achieved **>95% accuracy** on MNIST with simple 2-layer network
- GPU training is **WAY FASTER** than CPU (depending on hardware)


#### 📚 **Comparison: NumPy vs PyTorch**

| Aspect | NumPy Implementation | PyTorch Implementation |
|--------|---------------------|----------------------|
| **Gradients** | Manual computation | Automatic (autograd) |
| **Speed** | CPU only | GPU accelerated |
| **Code Length** | More verbose | More concise |
| **Flexibility** | Full control | High-level abstractions |
| **Learning Value** | Understand fundamentals | Production-ready code |
